In [1]:
import pandas as pd
import scipy.stats as stats
from scipy.stats import f_oneway
from scipy.stats import shapiro, levene

In [2]:
df = pd.read_excel(r'C:\Users\biyingling\OneDrive - Cook County Government\Racial Equity\Racial_Equity_Survey_23_Analysis.xlsx',sheet_name ='Clean_data_no_PD' )

In [3]:
df.columns

Index(['Unnamed: 0', 'ID', 'Start time', 'Completion time', 'Email', 'Name',
       'EID', 'Name Lookup', 'Office Lookup',
       'I understand terms and definitions related to race, equity and inclusive leadership',
       'I view human difference as positive and a reason for celebration',
       'I understand how systems impact individuals across differences in race, socioeconomic status, gender, gender expression and abilities.',
       'I understand government’s role in the day-to-day experiences of its residents',
       'I am aware of my how my cultural perspective influences my judgement on what are “appropriate” or “normal” behaviors, values and communication styles',
       'I can participate in discussions about race, equity and inclusion with relative ease',
       'I proactively engage in spaces and with people where I can learn about differences  and create relationships.',
       'I understand how my individual work connects to organizational equity work',
       'Resourc

In [6]:
df.drop(['ID', 'Email', 'Name', 'EID', 'Name Lookup', 'Office Lookup'], axis = 1, inplace =True)

In [7]:
questions = ['I understand terms and definitions related to race, equity and inclusive leadership',
       'I view human difference as positive and a reason for celebration',
       'I understand how systems impact individuals across differences in race, socioeconomic status, gender, gender expression and abilities.',
       'I understand government’s role in the day-to-day experiences of its residents',
       'I am aware of my how my cultural perspective influences my judgement on what are “appropriate” or “normal” behaviors, values and communication styles',
       'I can participate in discussions about race, equity and inclusion with relative ease',
       'I proactively engage in spaces and with people where I can learn about differences  and create relationships.',
       'I understand how my individual work connects to organizational equity work',
       'Resources and information needed to do my job with an equity lens are readily available to me.',
       'Partnerships with representatives from diverse communities (racial, ethnic, socioeconomic, People with Disabilities, LGBTQ) with their knowledge and experience are actively incorporated in our org...',
       'Cook County supports involvement with and/or utilization of regional and/or national forums that promote cultural competence.',
       'Training resources are available to support Cook County personnel to develop and grow in cultural competency',
       'I am proud to work at the county ',
       'I feel valued in my county position',
       'I can be my authentic self in my service at the county',
       'Perspectives like mine are considered and included in the decision making at the county',
       'Working at the county is important to the way I think of myself as a person']

In [8]:
df['Race/Ethnicity'].unique()

array(['Asian', 'Black/African American', 'White',
       'Hispanic/Latinx or Spanish origin', 'Prefer not to say',
       'Multiracial- two or more of the above', 'Other',
       'American Indian or Alaskan Native',
       'Native Hawaiian or Other Pacific Islander'], dtype=object)

In [ ]:
# Test if I can isolate each group's responses 
g_1 = df.groupby('Race/Ethnicity').get_group('Asian')
g_1

In [ ]:
# Preparing dataframes for ANOVA tests 
# Create a dictionary of dataframes, where each dataframe is the responses by each racial/
# ethnic group, and the key for each dataframe is the name of that group 
dataframes ={}
race = ['Asian', 'Black/African American', 'White',
       'Hispanic/Latinx or Spanish origin', 'Prefer not to say',
       'Multiracial- two or more of the above', 
       'Native Hawaiian or Other Pacific Islander']
for ethnicity in race:
    new_df = df.groupby('Race/Ethnicity').get_group(ethnicity)
    dataframes[ethnicity] = new_df
dataframes

In [ ]:
# Combining the entries for 'other','American Indians or Alaskan Native' 
# because there are too few data-points in these categories 

other = df.loc[(df['Race/Ethnicity'] == 'Other')|(df['Race/Ethnicity'] == 
                                                  'American Indian or Alaskan Native')]
dataframes['Others'] =other
dataframes

In [13]:
# For each survey question, create a new dictionary. This new dictionary contains the 
# responses of each racial/ethnic group to the n_th question. Previously, the dictionary 
# called "dataframes" contains all the responses of each ethnic group. Slicing dataframe[e][q] 
# creates a Series of all asians' responses to question 1, another Series of all whites' responses 
# to question 1, another Series of all blacks' responses to question 1, and so on. 
# Make these series into lists. 
# Now we have 3 lists to run f_oneway() on. Also check normality and variances. Print the results.
# Repeat this for each of the questions in the questions list.
col1 = []
col2 = []
col3 =[]
col4 =[]
col5 =[]
col6=[]
for i in range(0, len(questions)): 
    q = questions[i]
    q_df = {}
    n = {}
    for e in dataframes.keys():
        new_df = dataframes[e][q]
        q_df[e] = list(new_df.values)
        w, p_normality = shapiro(q_df[e])
        n[e] = p_normality 
    F, p = f_oneway(*q_df.values())
    if p < 0.05:
        significant = 'yes'
    else:
        significant ='no'    
#     print(q,'F-value:', F, 'p_value:', p, significant)
    col1.append(q)
    col2.append(F)
    col3.append(p)
    col4.append(significant)
    if all(value > 0.05 for value in n.values()):
        col5.append('Data for this question is normally distributed')
    else:
        col5.append('Data for this question is not normally distributed')
    W, p_2 = levene(*q_df.values())
    if p_2 > 0.05:
        col6.append('The variances for this question are equal')
    else:
        col6.append('The variances for this question are not equal')
results = pd.DataFrame({'Question':col1, 'F-value':col2, 'p-value(ANOVA)':col3, 'Significant':col4, 
           'Normality':col5, 'Variances':col6})
results

,Question,F-value,p-value(ANOVA),Significant,Normality,Variances
0,I understand terms and definitions related to ...,7.174223,2.428436e-08,yes,Data for this question is not normally distrib...,The variances for this question are equal
1,I view human difference as positive and a reas...,5.012322,1.423786e-05,yes,Data for this question is not normally distrib...,The variances for this question are equal
2,I understand how systems impact individuals ac...,5.458616,3.875644e-06,yes,Data for this question is not normally distrib...,The variances for this question are equal
3,I understand government’s role in the day-to-d...,6.928955,5.038922e-08,yes,Data for this question is not normally distrib...,The variances for this question are equal
4,I am aware of my how my cultural perspective i...,4.305161,1.090726e-04,yes,Data for this question is not normally distrib...,The variances for this question are equal
5,"I can participate in discussions about race, e...",6.515288,1.720944e-07,yes,Data for this question is not normally distrib...,The variances for this question are equal
6,I proactively engage in spaces and with people...,4.308050,1.081774e-04,yes,Data for this question is not normally distrib...,The variances for this question are equal
7,I understand how my individual work connects t...,3.954376,2.949052e-04,yes,Data for this question is not normally distrib...,The variances for this question are equal
8,Resources and information needed to do my job ...,2.761439,7.704474e-03,yes,Data for this question is not normally distrib...,The variances for this question are equal
9,Partnerships with representatives from diverse...,2.144102,3.709877e-02,yes,Data for this question is not normally distrib...,The variances for this question are not equal


In [14]:
# It looks like we cannot run ANOVA tests on this data, because the criterion of normality is not met. 
# Use Kruskal-Wallis test instead. 
col_1 = []
col_2 = []
col_3 = []
col_4 = [] 
for i in range(0, len(questions)): 
    q = questions[i]
    q_df = {}
    for e in dataframes.keys():
        new_df = dataframes[e][q]
        q_df[e] = list(new_df.values)
    S, pv = stats.kruskal(*q_df.values())
    if pv < 0.05:
        significant = 'yes'
    else:
        significant ='no'    
    col_1.append(q)
    col_2.append(S)
    col_3.append(pv)
    col_4.append(significant)
results_k = pd.DataFrame({'Question':col_1, 'Stat':col_2, 'p_value': col_3, 'Significance': col_4})
results_k

,Question,Stat,p_value,Significance
0,I understand terms and definitions related to ...,41.943640,5.331526e-07,yes
1,I view human difference as positive and a reas...,38.486278,2.448745e-06,yes
2,I understand how systems impact individuals ac...,34.754204,1.243895e-05,yes
3,I understand government’s role in the day-to-d...,39.822994,1.360881e-06,yes
4,I am aware of my how my cultural perspective i...,31.628205,4.759445e-05,yes
5,"I can participate in discussions about race, e...",44.444442,1.752765e-07,yes
6,I proactively engage in spaces and with people...,31.138883,5.861169e-05,yes
7,I understand how my individual work connects t...,27.876110,2.315531e-04,yes
8,Resources and information needed to do my job ...,19.597964,6.506785e-03,yes
9,Partnerships with representatives from diverse...,15.195522,3.357319e-02,yes


In [15]:
df['Gender'].unique()

array(['Male', 'Female', 'Prefer not to say', 'Nonbinary', 'Other'],
      dtype=object)

In [16]:
# Repeat the above operations on gender categories
dataframes_2 ={}
gender = ['Male', 'Female']
for g in gender:
    new_df_2 = df.groupby('Gender').get_group(g)
    dataframes_2[g] = new_df_2
dataframes_2['Others/Prefer Not to Say'] = df.loc[(df['Gender']=='Prefer not to say')|
                                                 (df['Gender']=='Nonbinary')|(df['Gender']=='Other')]
dataframes_2

{'Male':      Unnamed: 0          Start time     Completion time  \
 0             0 2022-12-29 09:47:13 2023-01-03 10:14:58   
 1             1 2023-01-13 12:41:24 2023-01-13 12:48:38   
 2             2 2023-01-11 12:43:28 2023-01-11 12:47:19   
 3             3 2022-12-16 09:35:18 2023-01-09 12:12:52   
 6             6 2023-01-09 08:32:04 2023-01-09 08:35:18   
 ..          ...                 ...                 ...   
 793         984 2022-12-27 10:41:18 2022-12-27 10:43:39   
 795         986 2022-12-27 10:13:56 2023-01-09 12:38:27   
 798         989 2023-01-11 13:39:46 2023-01-11 13:46:11   
 799         990 2023-01-09 12:22:54 2023-01-09 12:51:31   
 801         992 2023-01-10 10:53:17 2023-01-10 10:57:13   
 
      I understand terms and definitions related to race, equity and inclusive leadership  \
 0                                                  4.0                                     
 1                                                  1.0                             

In [17]:
c1 = []
c2 = []
c3 = []
c4 = []
c5 = []
c6 = []

for i in range(0, len(questions)): 
    q = questions[i]
    q_df = {}
    n = {}
    for g in dataframes_2.keys():
        new_df = dataframes_2[g][q]
        q_df[g] = list(new_df.values)
        w, p_normality = shapiro(q_df[g])
        n[g] = p_normality 
    F, p = f_oneway(*q_df.values())
    if p < 0.05:
        significant = 'yes'
    else:
        significant ='no'
    c1.append(q)
    c2.append(F)
    c3.append(p)
    c4.append(significant)    
    if all(value > 0.05 for value in n.values()):
        c5.append('Data for this question is normally distributed')
    else:
        c5.append('Data for this question is not normally distributed')
    W, p_2 = levene(*q_df.values())
    if p_2 > 0.05:
        c6.append('The variances for this question are equal')
    else:
        c6.append('The variances for this question are not equal')
results_2 = pd.DataFrame({'Question':c1, 'F-value':c2, 'p-value(ANOVA)':c3, 'Significant':c4, 
           'Normality':c5, 'Variances':c6})
results_2

,Question,F-value,p-value(ANOVA),Significant,Normality,Variances
0,I understand terms and definitions related to ...,17.829132,2.657723e-08,yes,Data for this question is not normally distrib...,The variances for this question are equal
1,I view human difference as positive and a reas...,13.922617,1.138501e-06,yes,Data for this question is not normally distrib...,The variances for this question are not equal
2,I understand how systems impact individuals ac...,14.227380,8.481557e-07,yes,Data for this question is not normally distrib...,The variances for this question are equal
3,I understand government’s role in the day-to-d...,4.262424,1.440941e-02,yes,Data for this question is not normally distrib...,The variances for this question are equal
4,I am aware of my how my cultural perspective i...,12.414941,4.900500e-06,yes,Data for this question is not normally distrib...,The variances for this question are equal
5,"I can participate in discussions about race, e...",4.458503,1.186884e-02,yes,Data for this question is not normally distrib...,The variances for this question are equal
6,I proactively engage in spaces and with people...,5.780122,3.218765e-03,yes,Data for this question is not normally distrib...,The variances for this question are equal
7,I understand how my individual work connects t...,11.448228,1.252905e-05,yes,Data for this question is not normally distrib...,The variances for this question are equal
8,Resources and information needed to do my job ...,3.335678,3.608613e-02,yes,Data for this question is not normally distrib...,The variances for this question are equal
9,Partnerships with representatives from diverse...,2.719047,6.654697e-02,no,Data for this question is not normally distrib...,The variances for this question are equal


In [20]:
c_1 = []
c_2 = []
c_3 = []
c_4 = [] 
for i in range(0, len(questions)): 
    q = questions[i]
    q_df = {}
    for g in dataframes_2.keys():
        new_df = dataframes_2[g][q]
        q_df[g] = list(new_df.values)
    S_2, pv_2 = stats.kruskal(*q_df.values())
    if pv < 0.05:
        significant = 'yes'
    else:
        significant ='no'    
    c_1.append(q)
    c_2.append(S_2)
    c_3.append(pv_2)
    c_4.append(significant)
results_k2 = pd.DataFrame({'Question':c_1, 'Stat':c_2, 'p_value': c_3, 'Significance': c_4})
results_k2

,Question,Stat,p_value,Significance
0,I understand terms and definitions related to ...,23.631214,7.388344e-06,yes
1,I view human difference as positive and a reas...,31.027179,1.830348e-07,yes
2,I understand how systems impact individuals ac...,28.323322,7.074061e-07,yes
3,I understand government’s role in the day-to-d...,8.712344,1.282740e-02,yes
4,I am aware of my how my cultural perspective i...,22.806387,1.115979e-05,yes
5,"I can participate in discussions about race, e...",9.484736,8.717976e-03,yes
6,I proactively engage in spaces and with people...,12.450814,1.978519e-03,yes
7,I understand how my individual work connects t...,21.715152,1.925815e-05,yes
8,Resources and information needed to do my job ...,7.718697,2.108173e-02,yes
9,Partnerships with representatives from diverse...,5.935049,5.143046e-02,yes


In [21]:
 
results_k.to_excel(r'C:\Users\biyingling\OneDrive - Cook County Government\Racial Equity\Racial_Equity_Survey_23_Analysis_Race.xlsx')
results_k2.to_excel(r'C:\Users\biyingling\OneDrive - Cook County Government\Racial Equity\Racial_Equity_Survey_23_Analysis_Gender.xlsx')